In [1]:
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from thundersvm import SVC
from sklearn.svm import SVC
import random
from numpy.random import choice

c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
os.listdir(os.path.join(os.getcwd(),'data','GPDSSyntheticSignatures4k'))

['data_features.csv', 'firmasSINTESISmanuscritas', 'visual_features.csv']

In [3]:
data_f = pd.read_csv(os.path.join(os.getcwd(),'data','GPDSSyntheticSignatures4k','data_features.csv'))
visual_f = pd.read_csv(os.path.join(os.getcwd(),'data','GPDSSyntheticSignatures4k','visual_features.csv'))

In [4]:
data_f.columns = ['path','user_id','sig_id','fakeness']
data_f.head()

,path,user_id,sig_id,fakeness
0,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,29,True
1,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,7,False
2,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,4,False
3,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,26,True
4,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,17,False


In [5]:
visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.0,0.0,2.086571,1.046970,2.012711,0.0,1.471289,0.0,0.0,1.308222,...,0.000000,2.560242,0.000000,0.000000,1.761090,2.054384,0.0,0.0,1.875244,1.555655
1,0.0,0.0,1.909776,1.295273,0.983222,0.0,0.708483,0.0,0.0,0.912948,...,0.000000,2.821531,0.000000,1.145785,2.461549,1.759992,0.0,0.0,2.124749,1.689826
2,0.0,0.0,1.989003,1.175972,1.191586,0.0,1.047355,0.0,0.0,1.942684,...,0.000000,2.109819,0.017639,0.462309,2.615123,1.609518,0.0,0.0,2.022851,1.887849
3,0.0,0.0,2.345391,1.077277,2.091065,0.0,1.449416,0.0,0.0,1.610568,...,0.000000,2.581949,0.000000,0.000000,1.917314,1.687836,0.0,0.0,2.093908,1.857081
4,0.0,0.0,0.980762,0.578907,0.712328,0.0,0.727942,0.0,0.0,1.355375,...,0.234534,2.123285,0.000000,1.186418,2.821104,1.428768,0.0,0.0,1.720274,1.232280


In [6]:
print(data_f.shape)
print(visual_f.shape)
user_count = len(data_f['user_id'].unique())
print(user_count, 'users are enrolled in the dataset')

(216000, 4)
(216000, 2048)
4000 users are enrolled in the dataset


In [7]:
user_id_list = np.arange(1,len(data_f['user_id'].unique())/5+1)
train_idx, test_idx = train_test_split(user_id_list, train_size=0.8, random_state=1923)

c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
# Filter train users and test users.
data_f_train = data_f.loc[data_f['user_id'].isin(train_idx)]
data_f_test = data_f.loc[data_f['user_id'].isin(test_idx)]
# Filter fakes and genuines.
data_f_train_gen = data_f_train.loc[data_f_train['fakeness']==False]
data_f_train_fake = data_f_train.loc[data_f_train['fakeness']==True]
data_f_train_gen_5 = data_f_train_gen.loc[data_f_train_gen['sig_id'].isin(np.arange(1,6))]
data_f_train_gen_14 = data_f_train_gen.loc[data_f_train_gen['sig_id'].isin(np.arange(1,15))]
data_f_train_gen_valid_10 = data_f_train_gen.loc[data_f_train_gen['sig_id'].isin(np.arange(15,25))]

In [17]:
visual_f.loc[data_f_train_gen_14.loc[choice(data_f_train_gen_14.loc[data_f_train_gen_14['user_id']!=3].index,10)].index]

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
192695,0.0,0.000000,1.621573,0.752438,2.340590,0.0,1.527919,0.000000,0.000000,1.257096,...,0.000000,2.218231,0.000000,0.000000,1.447714,1.668069,0.0,0.0,1.279251,1.876335
58316,0.0,0.000000,0.271409,0.751206,0.093431,0.0,0.000000,0.703187,0.164419,1.222482,...,1.553104,0.302686,1.128608,0.551534,0.594208,0.471504,0.0,0.0,0.000000,1.883980
96752,0.0,0.085561,1.380951,1.273209,1.493375,0.0,0.735762,0.000000,0.000000,1.854687,...,0.000000,2.784135,0.000000,0.617424,2.019567,1.972065,0.0,0.0,2.209384,1.378371
60337,0.0,0.000000,1.298896,1.420663,1.260204,0.0,1.037401,0.000000,0.000000,1.711602,...,0.000000,2.634600,0.000000,1.421359,2.719750,1.646291,0.0,0.0,2.442997,1.877545
83234,0.0,0.467607,1.683026,2.387130,1.343066,0.0,1.613865,0.036213,0.000000,1.325640,...,0.233328,1.542116,0.000000,1.004324,1.391003,0.943640,0.0,0.0,1.740785,1.858059
60964,0.0,0.000000,1.722106,1.071501,2.263459,0.0,1.585181,0.000000,0.000000,1.239535,...,0.000000,2.448182,0.000000,0.000000,1.615686,1.517259,0.0,0.0,1.544139,1.738828
75217,0.0,0.137430,1.912181,0.896749,2.033177,0.0,1.603356,0.000000,0.000000,1.511271,...,0.000000,2.486612,0.000000,0.000000,1.636138,1.752387,0.0,0.0,1.690389,1.911143
49428,0.0,0.060129,1.418332,1.205353,2.070050,0.0,0.956670,0.000000,0.000000,1.131921,...,0.000000,1.794722,0.000000,0.000000,1.520536,1.791769,0.0,0.0,1.421181,2.156775
185165,0.0,0.000000,1.832127,1.112945,1.933915,0.0,1.433295,0.000000,0.000000,1.153206,...,0.000000,2.862256,0.000000,0.250512,2.097370,1.960247,0.0,0.0,2.131303,2.134420
12702,0.0,0.000000,2.013968,1.294413,1.719007,0.0,1.249856,0.000000,0.000000,2.108005,...,0.000000,2.412355,0.000000,0.129142,1.751121,1.619020,0.0,0.0,1.863611,1.954757


In [20]:
data_f_train_gen_14.loc[choice(data_f_train_gen_14.loc[data_f_train_gen_14['user_id']!=3].index,10)]

,path,user_id,sig_id,fakeness
212035,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,298,6,False
74093,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,790,8,False
33929,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,88,13,False
12687,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,613,3,False
95874,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,273,5,False
183784,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,782,14,False
104810,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,140,3,False
74092,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,790,12,False
145167,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,54,4,False
169193,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,522,12,False


In [12]:
# choice(data_f_train_gen_14.loc[data_f_train_gen_14['user_id']!=3].index,10)

In [23]:
scores_sum = 0
run_count = 0
for user_id in tqdm.tqdm(data_f_train_gen_14['user_id'].unique()):
    y_train = data_f_train_gen_14['user_id']==user_id
    X_train = visual_f.loc[y_train.index]
    # clf = XGBClassifier(tree_method='gpu_hist',verbosity=2,predictor='gpu_predictor', scale_pos_weight=8946/14)
    clf = SVC(class_weight='balanced', gamma='scale')
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_train_fake.loc[data_f_train_fake['user_id']==user_id]
    y_valid = len(y_valid_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_fakes.index]
    scores_sum += clf.score(X_valid,y_valid)
    run_count += 1
    y_valid_gens = data_f_train_gen_valid_10.loc[data_f_train_gen_valid_10['user_id']==user_id]
    y_valid = len(y_valid_gens.index)*[True]
    X_valid = visual_f.loc[y_valid_gens.index]
    scores_sum += clf.score(X_valid,y_valid)
    run_count += 1
    y_valid_rand_fakes = data_f_train_gen_14.loc[choice(data_f_train_gen_14.loc[data_f_train_gen_14['user_id']!=3].index,10)]
    y_valid = len(y_valid_rand_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_rand_fakes.index]
    scores_sum += clf.score(X_valid,y_valid)
    run_count += 1
    print(scores_sum/run_count)
print(scores_sum/run_count)



  0%|                                                                                          | 0/640 [00:00<?, ?it/s]

0.8666666666666667




  0%|▏                                                                               | 1/640 [00:16<3:00:49, 16.98s/it]

0.8722222222222222




  0%|▎                                                                               | 2/640 [00:36<3:08:21, 17.71s/it]

...

0.843589743589744




 10%|████████                                                                       | 65/640 [20:32<2:41:00, 16.80s/it]

0.8432659932659937




 10%|████████▏                                                                      | 66/640 [20:58<3:08:21, 19.69s/it]

KeyboardInterrupt: 